In [2]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 5.1 MB/s eta 0:00:00


In [3]:
# ==========================
# Import Libraries
# ==========================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from category_encoders import TargetEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# ==========================
# Load Dataset
# ==========================
data = pd.read_csv('kc_house_data.csv')

# ==========================
# Drop Unnecessary Columns
# ==========================
drop_cols = ['date', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']
data = data.drop(columns=drop_cols)

# ==========================
# Feature Engineering
# ==========================
data['basement_exists'] = data['sqft_basement'].apply(lambda x: 1 if x > 0 else 0)
median_nonzero = data.loc[data['sqft_basement'] > 0, 'sqft_basement'].median()
data['sqft_basement_imputed'] = data['sqft_basement'].replace(0, median_nonzero)

ordinal_cols = ['condition', 'grade']
le = LabelEncoder()
for col in ordinal_cols:
    data[col] = le.fit_transform(data[col])

# ==========================
# Log-transform Target
# ==========================
data['price_log'] = np.log1p(data['price'])

numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns.tolist()
numeric_cols.remove('price')
numeric_cols.remove('price_log')

X = data[numeric_cols]
y = data['price_log']

# ==========================
# Train-Validation-Test Split
# ==========================
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42)

# ==========================
# Target Encoding for 'zipcode'
# ==========================
encoder = TargetEncoder(cols=['zipcode'])
encoder.fit(X_train['zipcode'], y_train)

X_train['zipcode'] = encoder.transform(X_train['zipcode'])
X_val['zipcode'] = encoder.transform(X_val['zipcode'])
X_test['zipcode'] = encoder.transform(X_test['zipcode'])

# ==========================
# Scaling (optional for RF)
# ==========================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)


#ANN :

In [4]:

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1,1)
X_val_tensor = torch.tensor(X_val_scaled, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32).view(-1,1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1,1)

# ==========================
# DataLoaders
# ==========================
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=32)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32)

# ==========================
# ANN Model with BatchNorm + Dropout
# ==========================
class ImprovedANN(nn.Module):
    def __init__(self, input_dim):
        super(ImprovedANN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,1)
        )
    def forward(self, x):
        return self.model(x)

input_dim = X_train_scaled.shape[1]
model = ImprovedANN(input_dim)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10)

# ==========================
# Train Model
# ==========================
num_epochs = 200
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * X_batch.size(0)

    epoch_train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(epoch_train_loss)

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            val_loss += loss.item() * X_batch.size(0)
    epoch_val_loss = val_loss / len(val_loader.dataset)
    val_losses.append(epoch_val_loss)

    scheduler.step(epoch_val_loss)  # reduce lr if val_loss plateaus

    if (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] Train Loss: {epoch_train_loss:.4f} Val Loss: {epoch_val_loss:.4f}")

# ==========================
# Evaluate Model
# ==========================
model.eval()
with torch.no_grad():
    y_train_pred = np.expm1(model(X_train_tensor).numpy())
    y_val_pred = np.expm1(model(X_val_tensor).numpy())
    y_test_pred = np.expm1(model(X_test_tensor).numpy())

from sklearn.metrics import mean_squared_error, r2_score

print("Training MSE:", mean_squared_error(np.expm1(y_train), y_train_pred))
print("Validation MSE:", mean_squared_error(np.expm1(y_val), y_val_pred))
print("Test MSE:", mean_squared_error(np.expm1(y_test), y_test_pred))
print("Training R2:", r2_score(np.expm1(y_train), y_train_pred))
print("Validation R2:", r2_score(np.expm1(y_val), y_val_pred))
print("Test R2:", r2_score(np.expm1(y_test), y_test_pred))

Epoch [10/200] Train Loss: 0.2524 Val Loss: 0.0526
Epoch [20/200] Train Loss: 0.0634 Val Loss: 0.0385
Epoch [30/200] Train Loss: 0.0498 Val Loss: 0.0376
Epoch [40/200] Train Loss: 0.0449 Val Loss: 0.0368
Epoch [50/200] Train Loss: 0.0416 Val Loss: 0.0379
Epoch [60/200] Train Loss: 0.0416 Val Loss: 0.0332
Epoch [70/200] Train Loss: 0.0406 Val Loss: 0.0349
Epoch [80/200] Train Loss: 0.0371 Val Loss: 0.0321
Epoch [90/200] Train Loss: 0.0362 Val Loss: 0.0335
Epoch [100/200] Train Loss: 0.0355 Val Loss: 0.0318
Epoch [110/200] Train Loss: 0.0342 Val Loss: 0.0314
Epoch [120/200] Train Loss: 0.0330 Val Loss: 0.0312
Epoch [130/200] Train Loss: 0.0338 Val Loss: 0.0315
Epoch [140/200] Train Loss: 0.0326 Val Loss: 0.0313
Epoch [150/200] Train Loss: 0.0332 Val Loss: 0.0311
Epoch [160/200] Train Loss: 0.0325 Val Loss: 0.0314
Epoch [170/200] Train Loss: 0.0323 Val Loss: 0.0311
Epoch [180/200] Train Loss: 0.0324 Val Loss: 0.0312
Epoch [190/200] Train Loss: 0.0323 Val Loss: 0.0309
Epoch [200/200] Train

In [13]:
model

ImprovedANN(
  (model): Sequential(
    (0): Linear(in_features=17, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): ReLU()
    (10): Linear(in_features=64, out_features=1, bias=True)
  )
)



## Artificial Neural Network (ANN) Architecture  
This model is a **deep feed-forward neural network** designed for regression.  
It integrates **Batch Normalization**, **Dropout**, and a **Learning Rate Scheduler** to improve stability and prevent overfitting.

### Layer Structure
1. **Input Layer**
   - `input_dim` features (same as scaled dataset)

2. **Hidden Layer 1**
   - Linear: **input_dim → 256**
   - BatchNorm1d(256)
   - ReLU activation
   - Dropout(0.3)

3. **Hidden Layer 2**
   - Linear: **256 → 128**
   - BatchNorm1d(128)
   - ReLU activation
   - Dropout(0.2)

4. **Hidden Layer 3**
   - Linear: **128 → 64**
   - ReLU activation

5. **Output Layer**
   - Linear: **64 → 1**
   - Returns the regression prediction

---

## Key Hyperparameters

### **Optimizer: Adam**
- Learning rate: **0.001**
- Beta1, Beta2 defaults
- Works well for noisy, non-linear regression tasks

### **Loss Function: MSELoss**
- Suitable for continuous numeric targets  
- Model optimizes mean squared error between predictions & true values

### **Learning Rate Scheduler**
`ReduceLROnPlateau`  
- factor = **0.5**  
- patience = **10 epochs**  
- Reduces LR when validation loss stops improving  
→ helps escape plateaus & improves convergence

### **Batch Size**
- **32**
- Balanced for speed + gradient stability

### **Dropout**
- Prevents overfitting
- Dropout rates:
  - 0.3 in first layer
  - 0.2 in second layer

### **Batch Normalization**
- Stabilizes activations  
- Allows higher learning rates  
- Improves gradient flow  

---

## Training Details
- Total epochs: **200**
- Training performed using **DataLoader** on shuffled batches
- Validation loss monitored each epoch
- LR scheduler adjusts learning rate dynamically  
- Model switches between:
  - `model.train()` for gradient updates  
  - `model.eval()` for validation  

---

## Evaluation Details
After training, predictions are:
- Converted back from the log scale using **expm1()**
- Evaluated on:
  - **Training**
  - **Validation**
  - **Test** sets

Metrics computed:
- **Mean Squared Error (MSE)**
- **R² Score**
- Performance reported separately for train/val/test




#Random Forest :

In [5]:

# ==========================
# Random Forest Model
# ==========================
rf = RandomForestRegressor(
    n_estimators=301,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)

# Train
rf.fit(X_train_scaled, y_train)

# Predict
y_train_pred = np.expm1(rf.predict(X_train_scaled))
y_val_pred = np.expm1(rf.predict(X_val_scaled))
y_test_pred = np.expm1(rf.predict(X_test_scaled))

# ==========================
# Results
# ==========================
print("Training MSE:", mean_squared_error(np.expm1(y_train), y_train_pred))
print("Validation MSE:", mean_squared_error(np.expm1(y_val), y_val_pred))
print("Test MSE:", mean_squared_error(np.expm1(y_test), y_test_pred))

print("Training R2:", r2_score(np.expm1(y_train), y_train_pred))
print("Validation R2:", r2_score(np.expm1(y_val), y_val_pred))
print("Test R2:", r2_score(np.expm1(y_test), y_test_pred))


Training MSE: 3053770088.756851
Validation MSE: 14948286152.595942
Test MSE: 18118163837.813744
Training R2: 0.976805045258336
Validation R2: 0.8862059427359875
Test R2: 0.8812852373775151


In [6]:
rf

RandomForestRegressor(n_estimators=301, n_jobs=-1, random_state=42)

# Specify Architecture, Hyperparameters, Training Details

## Random Forest Architecture
- Ensemble model using **Bagging (Bootstrap Aggregation)**  
- Contains multiple **independent Decision Trees**  
- **Final prediction = average** of all tree predictions  

---

## Key Hyperparameters

### `n_estimators = 301`
- Number of trees in the forest  
- More trees → better accuracy, but slower training  

### `max_depth = None`
- Trees grow until all leaves are pure  
- No maximum depth limit  

### `min_samples_split = 2`
- Minimum number of samples needed to split a node  

### `min_samples_leaf = 1`
- Minimum samples required in a leaf node  

### `random_state = 42`
- Ensures reproducible results  

### `n_jobs = -1`
- Uses **all CPU cores** for parallel training  

---

## Training Details
- Each tree is trained on a **bootstrap sample**  
- Each split considers a **random subset of features** → reduces overfitting  
- **Feature scaling is not required**, but included for consistency  
- Learns **non-linear relationships** extremely well  
- Very robust to **noise and outliers**  

---

## Evaluation Metrics
- **MSE** (Mean Squared Error)  
- **R² Score** on Train / Validation / Test  
- **Feature Importance Plot** for model interpretation  


#Arbre of decision :

In [8]:
from sklearn.tree import DecisionTreeRegressor, plot_tree
dt_model = DecisionTreeRegressor(
    random_state=42,
    max_depth=14,           # Maximum tree depth
    min_samples_split=20,   # Minimum samples required to split a node
    min_samples_leaf=10     # Minimum samples required at a leaf node
)

dt_model.fit(X_train, y_train)

# ==========================
# Predictions
# ==========================
y_train_pred = dt_model.predict(X_train)
y_val_pred = dt_model.predict(X_val)
y_test_pred = dt_model.predict(X_test)

# ==========================
# Evaluate Model
# ==========================
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Validation MSE:", mean_squared_error(y_val, y_val_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))
print("Training R2:", r2_score(y_train, y_train_pred))
print("Validation R2:", r2_score(y_val, y_val_pred))
print("Test R2:", r2_score(y_test, y_test_pred))


Training MSE: 0.024325122862176165
Validation MSE: 0.04243768150009057
Test MSE: 0.0459539930751791
Training R2: 0.9113605761376141
Validation R2: 0.8484492289106691
Test R2: 0.8406322275275888


In [11]:
dt_model


DecisionTreeRegressor(max_depth=14, min_samples_leaf=10, min_samples_split=20,
                      random_state=42)


## Decision Tree Regressor Architecture
A **Decision Tree Regressor** predicts continuous values by recursively splitting the dataset into regions with similar target values.

### How It Works
- The tree chooses the **best feature + split point** to reduce prediction error.
- Splits continue recursively until stopping criteria are reached.
- Final prediction = **mean of samples in the leaf node**.

This model is simple, interpretable, and captures **non-linear relationships** without requiring feature scaling.

---

## Key Hyperparameters

### `max_depth = 14`
- Maximum depth (levels) of the tree  
- Controls how complex the tree can grow  
→ **Higher depth → more learning capacity, but more overfitting**

### `min_samples_split = 20`
- Minimum number of samples required to split a node  
→ Prevents the tree from growing too deep on noisy data

### `min_samples_leaf = 10`
- Minimum samples allowed in a leaf node  
→ Makes model more stable and reduces variance

### `random_state = 42`
- Ensures reproducible results

---

## Training Details
- Model is trained on full **X_train, y_train** without batching  
- Decision trees learn by:
  - Greedy splitting (best local split)
  - Minimizing **MSE impurity**
- No feature scaling required (trees are scale-invariant)
- Faster training compared to ensemble models or neural networks

---

## Evaluation Details
Predictions are made on:
- Training set  
- Validation set  
- Test set  

Metrics computed:
- **MSE** (Mean Squared Error)
- **R² Score**

These allow checking:
- If the model is overfitting  
- Generalization on unseen data  


#SVR :

In [10]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# ==========================
# Scaling (important for SVR)
# ==========================
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled   = scaler_X.transform(X_val)
X_test_scaled  = scaler_X.transform(X_test)

# FIX: reshape using values
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()

# ==========================
# SVR Model
# ==========================
svr_model = SVR(
    kernel="rbf",
    C=10,
    epsilon=0.1,
    gamma="scale"
)

svr_model.fit(X_train_scaled, y_train_scaled)

# ==========================
# Predictions (inverse transform)
# ==========================
y_train_pred = scaler_y.inverse_transform(
    svr_model.predict(X_train_scaled).reshape(-1, 1)
)

y_val_pred = scaler_y.inverse_transform(
    svr_model.predict(X_val_scaled).reshape(-1, 1)
)

y_test_pred = scaler_y.inverse_transform(
    svr_model.predict(X_test_scaled).reshape(-1, 1)
)

# ==========================
# Evaluate Model
# ==========================
print("Training MSE:", mean_squared_error(y_train, y_train_pred))
print("Validation MSE:", mean_squared_error(y_val, y_val_pred))
print("Test MSE:", mean_squared_error(y_test, y_test_pred))

print("Training R2:", r2_score(y_train, y_train_pred))
print("Validation R2:", r2_score(y_val, y_val_pred))
print("Test R2:", r2_score(y_test, y_test_pred))


Training MSE: 0.020940957071808024
Validation MSE: 0.03598603905840534
Test MSE: 0.03755940910060124
Training R2: 0.9236922921010907
Validation R2: 0.871488927411351
Test R2: 0.8697445213531207


In [12]:
svr_model

SVR(C=10)


## Support Vector Regression (SVR) Architecture
Support Vector Regression is a powerful model based on **Support Vector Machines**, adapted for continuous outputs.

### How SVR Works
- Tries to fit a curve within an **epsilon margin** around the data.
- Only points **outside the margin** influence the model → called **support vectors**.
- Uses kernel functions (e.g., RBF) to model **non-linear relationships**.

SVR is effective for small-to-medium datasets and excels when features are properly scaled.

---

## Feature Scaling (Very Important for SVR)
Because SVR is distance-based, scaling is required:

- `StandardScaler()` applied to **X**  
- `StandardScaler()` applied to **y**  
- Predictions are inverse-transformed back to original scale.

---

## Key Hyperparameters

### `kernel = "rbf"`
- Radial Basis Function kernel  
- Allows the model to learn **complex non-linear patterns**

### `C = 10`
- Regularization parameter  
- Higher C → model tries to fit training data more strictly  
- Lower C → smoother, more generalized model

### `epsilon = 0.1`
- Width of epsilon-tube  
- Errors inside the tube are ignored  
→ Controls sensitivity to small variations in target values

### `gamma = "scale"`
- Kernel coefficient  
- Determines how far influence of a single sample reaches  
- `"scale"` adjusts automatically based on data variance

---

## Training Details
- Model trained on **scaled inputs & outputs**  
- Uses **Quadratic Programming Solver** (slower than trees/ANNs but very precise)
- No batching (SVR trains on full dataset)

### SVR Strengths
- Excellent for smooth regression curves  
- Robust to outliers due to epsilon margin  
- Good for non-linear patterns with limited data

---

## Evaluation Details
Predictions are inverse-transformed using the target scaler.

Metrics for Train / Validation / Test:
- **MSE** (Mean Squared Error)  
- **R² Score**  

Useful to check:
- Overfitting (Train vs Val/Test performance)  
- Model generalization  


